# Voxelization

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import IPython.display
import PIL.Image
import random

test_data_dir = "./test_data"

#### Visualization Configurations

In [2]:
# change to True if you want to interact with the visualization windows
#interactive = not "CI" in os.environ
interactive = True
def jupyter_draw_geometries(
    geoms,
    window_name="Open3D",
    width=1920,
    height=1080,
    left=50,
    top=50,
    point_show_normal=False,
    mesh_show_wireframe=False,
    mesh_show_back_face=False,
    lookat=None,
    up=None,
    front=None,
    zoom=None,
):
    vis = o3d.visualization.Visualizer()
    vis.create_window(
        window_name=window_name,
        width=width,
        height=height,
        left=left,
        top=top,
        visible=True,  # If false, capture_screen_float_buffer() won't work.
    )
    vis.get_render_option().point_show_normal = point_show_normal
    vis.get_render_option().mesh_show_wireframe = mesh_show_wireframe
    vis.get_render_option().mesh_show_back_face = mesh_show_back_face
    for geom in geoms:
        vis.add_geometry(geom)
    if lookat is not None:
        vis.get_view_control().set_lookat(lookat)
    if up is not None:
        vis.get_view_control().set_up(up)
    if front is not None:
        vis.get_view_control().set_front(front)
    if zoom is not None:
        vis.get_view_control().set_zoom(zoom)
    if interactive:
        vis.run()
    else:
        for geom in geoms:
            vis.update_geometry(geom)
        vis.poll_events()
        vis.update_renderer()
    im = vis.capture_screen_float_buffer()
    vis.destroy_window()
    im = (255 * np.asarray(im)).astype(np.uint8)
    IPython.display.display(PIL.Image.fromarray(im, "RGB"))

o3d.visualization.draw_geometries = jupyter_draw_geometries

## Point Cloud

In [3]:
N = 1000
voxel_size = 0.2
pcd_path = test_data_dir + "/cloud_bin_0.ply"
pcd = o3d.io.read_point_cloud(pcd_path)

"""
if not os.path.exists(pcd_path):
    print(f"point cloud not exist in path: {pcd_path}")
else:
    # fit to unit cube
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
              center=pcd.get_center())
    pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
    o3d.visualization.draw_geometries([pcd])
"""

'\nif not os.path.exists(pcd_path):\n    print(f"point cloud not exist in path: {pcd_path}")\nelse:\n    # fit to unit cube\n    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),\n              center=pcd.get_center())\n    pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))\n    o3d.visualization.draw_geometries([pcd])\n'

#### Downsample

In [4]:
downpcd = pcd.voxel_down_sample(voxel_size=voxel_size)
"""
for point in np.asarray(downpcd.points):
    print(point)

"""

"""
out1, out2, out3 = pcd.voxel_down_sample_and_trace(voxel_size=voxel_size, min_bound=pcd.get_min_bound(), max_bound=pcd.get_max_bound())

print(out1)
print(out2[2])
print(out3[0].pop())
"""

"""
# TEST
pcdarray = np.asarray(pcd.points)
downarray = np.asarray(downpcd.points)

print(len(pcdarray))
print(out1.points[0])
print(pcd.points[out3[0][0]])

#o3d.visualization.draw_geometries([downpcd])
"""


'\n# TEST\npcdarray = np.asarray(pcd.points)\ndownarray = np.asarray(downpcd.points)\n\nprint(len(pcdarray))\nprint(out1.points[0])\nprint(pcd.points[out3[0][0]])\n\n#o3d.visualization.draw_geometries([downpcd])\n'

### Cube Type

In [5]:
class Cube(object):
    def __init__(self, range):
        """
        Builds a cube from x, y and z ranges
        """
        self.range = range
    """
    @classmethod
    def from_points(cls, firstcorner, secondcorner):
        """"""
        Builds a cube from the bounding points
        Rectangle.from_points(Point(0, 10, -10),
                              Point(10, 20, 0)) == Rectangle((0, 10), (10, 20), (-10, 0))
        """"""
    
        x = (a[:, None] < b).all(-1)
        return cls(*zip(firstcorner, secondcorner))
    """
    @classmethod
    def from_voxel_size(cls, center, voxel_size):
        """
        Builds a cube from the voxel size and center of the voxel
        """
        cls.center = center
        half_center = voxel_size / 2
        x_range = (center[0] - half_center, center[0] + half_center)
        y_range = (center[1] - half_center, center[1] + half_center)
        z_range = (center[2] - half_center, center[2] + half_center)
        range = np.array([[x_range[0], x_range[1], y_range[0] ,y_range[1] ,z_range[0], z_range[1]]])
        return cls(range)

    def contains_points(self, p):
        """
        Returns given point is in cube
        """
        less = np.repeat(self.range, repeats=[p.shape[0]], axis=0)[:, 0::2] < p
        greater = np.repeat(self.range, repeats=[p.shape[0]], axis=0)[:, 1::2] > p
        filter = np.logical_and(less.all(axis=1), greater.all(axis=1))
        return p[filter]
        
def get_random_color():
    return [round(random.uniform(0.0, 1.0), 1), 
    round(random.uniform(0.0, 1.0), 1), 
    round(random.uniform(0.0, 1.0), 1)]

cube = Cube.from_voxel_size((5, 15, -5), 10)
points = np.array([[3, 15, -8], [11, 15, -8]])

cube.contains_points(points)


array([[ 3, 15, -8]])

#### Search Points Inside the Voxel

In [20]:

pcds = []
for i in range(1,29):
    path = f'./test_data/Files/12_12_14_L4_V{i}.txt'
    print(path)
    arr_data = np.genfromtxt(path, delimiter=',')
    print(arr_data.shape)
    pcd_voxel = o3d.geometry.PointCloud()
    pcd_voxel.points = o3d.utility.Vector3dVector(arr_data[:,:3])
    pcd_voxel.paint_uniform_color(get_random_color())
    pcds.append(pcd_voxel)

o3d.visualization.draw_geometries(pcds)
o3d.io.write_point_clouds("./data.ply", pcds)

./test_data/Files/12_12_14_L4_V1.txt
(1549, 4)
./test_data/Files/12_12_14_L4_V2.txt
(1661, 4)
./test_data/Files/12_12_14_L4_V3.txt
(1789, 4)
./test_data/Files/12_12_14_L4_V4.txt
(1851, 4)
./test_data/Files/12_12_14_L4_V5.txt
(1911, 4)
./test_data/Files/12_12_14_L4_V6.txt
(2040, 4)
./test_data/Files/12_12_14_L4_V7.txt
(2059, 4)
./test_data/Files/12_12_14_L4_V8.txt
(2410, 4)
./test_data/Files/12_12_14_L4_V9.txt
(2525, 4)
./test_data/Files/12_12_14_L4_V10.txt
(2302, 4)
./test_data/Files/12_12_14_L4_V11.txt
(2540, 4)
./test_data/Files/12_12_14_L4_V12.txt
(2639, 4)
./test_data/Files/12_12_14_L4_V13.txt
(2582, 4)
./test_data/Files/12_12_14_L4_V14.txt
(2641, 4)
./test_data/Files/12_12_14_L4_V15.txt
(2527, 4)
./test_data/Files/12_12_14_L4_V16.txt
(2767, 4)
./test_data/Files/12_12_14_L4_V17.txt
(2581, 4)
./test_data/Files/12_12_14_L4_V18.txt
(2637, 4)
./test_data/Files/12_12_14_L4_V19.txt
(2792, 4)
./test_data/Files/12_12_14_L4_V20.txt
(2537, 4)
./test_data/Files/12_12_14_L4_V21.txt
(2521, 4)
.

TypeError: write_point_cloud(): incompatible function arguments. The following argument types are supported:
    1. (filename: str, pointcloud: open3d.cpu.pybind.geometry.PointCloud, write_ascii: bool = False, compressed: bool = False, print_progress: bool = False) -> bool

Invoked with: './data.ply', [PointCloud with 1549 points., PointCloud with 1661 points., PointCloud with 1789 points., PointCloud with 1851 points., PointCloud with 1911 points., PointCloud with 2040 points., PointCloud with 2059 points., PointCloud with 2410 points., PointCloud with 2525 points., PointCloud with 2302 points., PointCloud with 2540 points., PointCloud with 2639 points., PointCloud with 2582 points., PointCloud with 2641 points., PointCloud with 2527 points., PointCloud with 2767 points., PointCloud with 2581 points., PointCloud with 2637 points., PointCloud with 2792 points., PointCloud with 2537 points., PointCloud with 2521 points., PointCloud with 2410 points., PointCloud with 2222 points., PointC

In [6]:
pcdarray = np.asarray(pcd.points)
downarray = np.asarray(downpcd.points)

voxels = [Cube.from_voxel_size(center, voxel_size) for center in downarray]
point_inside = []

count = 0
pcds = []
for voxel in voxels:
    pcd_voxel = o3d.geometry.PointCloud()
    pcd_voxel.points = o3d.utility.Vector3dVector(voxel.contains_points(pcdarray))
    pcd_voxel.paint_uniform_color(get_random_color())
    count += len(pcd_voxel.points)
    pcds.append(pcd_voxel)

print("Total Points:", pcdarray.shape[0])
print("Count:", count)
o3d.visualization.draw_geometries(pcds)


#### Test Voxels

#### Voxel Visualization